# Instructions for main testing

### Running the main program
 The main function is defined in the Main.scala. The way you run this depends on whether you are using sbt alone or intellij
 

We are using intellij so choose run main configuration then press the play button


### Running ScalaTest tests

We will use a powerful unit testing package called scalatest. Just use the "Run Scala Tests" configuration.


## Instructions for submission
__Makes sure everthing is saved__ 

1. run all tests one last time to ensure that everything is working as expected. You will not be able to submit a code that doesn't at least compile


3. Run the __checkAndZipSubmission__ sbt task
    
        - Select "Zip Submission" configuration from the dropdown menu then click the run button (play button) 
        

4. Ensure whichever option you chosed dispays __[success]__ at the end 
5. Upload the generated "submission.zip" file located at the project root folder



# The Task
    
    - The objective of this project is to explore compiling this intoa stack machine bytecode and writing an emulator to execute this bytecode
    
    # The Language
We have encountered arithmetic expressions given by the grammar 


$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& | & Plus( \textbf{Expr}, \textbf{Expr})  \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{Identifier} & \rightarrow & [a-zA-Z][a-z\ A-Z\ 0-9\ \_]* & \text{Note: All strings that begin with letters}\\
&&& \text{a-z or A-Z and subsequently can contain a-z, A-Z, 0-9 or _ chars}
\end{array}$$

__Note__ No identifiers are included in this grammar. This is intentional.


# Part 1

Imagine a stack data structure that contains double precision numbers and the following instructions that operate on the stack. pay attention to the __SubI__ and __DivI__ instructions since the order matters. 


1.  __PushI(d):__ push the number d onto the stack


2.  __PopI:__ pop off the top element of the stack
            - throw an exception if the stack is empty 
            
3.  __AddI:__ pop two numbers off of the stack, then add them, then push the result back onto the stack
            - throw an exception if stack is empty during any of the pop operations
            
4.  __SubI:__ pop two numbers off of the stack 
            
                - let the first number of the popped be v1 and the second number be equal to v2, subtract them as v2 - v1 (this order is very important) and push the result back on to the stack. Throw an exception if the stack is empty during any of the pop operations        
                
                
5.  __MultI:__ pop two numbers off of the stack, multiply them, then push the result back onto the stack. Throw an exception if the stack is empty during any of these operations


6.  __DivI:__ pop two numbers from the stack, let the first number popped be v1 and the second number be v2, subtrac tthem as v2 / v1 (this order is very important) and push the result back to the stack. Throw and exception if the stack is empty during any of the pop operations. 


7.  __Log:__ pop one number from the stack, compute its log if positive and push the result back onto the stack. Throw an exception if the stack is empty during any of the pop operations

8.  __SineI/CosineI:__ pop one number from the stack, compute its sin/cos respectively, and push the result back onto the stack. Throw and exception if the stack is empty during any of the pop operations 



Given a sequence of instructions (eg., PushI(2.0), PushI(3.0), AddI, PushI(4.0), SubI], we execute each instruction in turn starting from the empty stack. We will impelment the stack as a list with the head of the list as the top of the stack

    * Initially the stack is empty. 
    * When we execute PushI(2.0), the stack is [2.0]
    * When we execute PushI(3.0), the stack becomes [3.0,2.0]
    * When we execute AddI, the stack becomes [5.0]
    * When we execute Push[4.0], the stack becomes [4.0 , 5.0]
    * When we execute SubI, the stack becomes [1.0]
            so that means v1 = 4.0 and v2 = 5.0
                    __THIS IMPLIES THE TOP OF THE STACK IS THE LAST ELEMENT ADDED__
                    
                    
                    
                    
                    
## Instructions for Part 1 

Implement the methods __emulateSingleInstruction__ and __emulateStackMachine__ in the file __StackMachineEmulator.scala__. For your convienience, the stack machine instructions have been defined as a very simple inductive defintion giving case classes for all instructions. We will use an __immutable__ List data structure to simulate the stack 

    * emulateSingleInstruction(stack, instruction) takes a stack which is a List of Double, the instruction which is an instruction. It returns a stack of double resulting from executing the given instructions
    
    * immulateStackmachine(listOfInstructions) , this function asks you to return the final value computed by the listOfinstructions. You should call emeulateSingleInstruction repeatedly. Use __foldLeft__ over lists instead of a loop
    
            essentially, just put the emulateSingleInstruction as the function arguemnt when you are going to use foldLeft

## See if you can write a foldLeft function that appends to the end of a list that are also type list

In [5]:
def AppendOnTopOfList(currentList: List[Int], y: Int): List[Int] = {
    val newey = currentList(0)
    
    currentList ::: List(newey) // merges from from this joint 
}

val myList = List(1,2,3,4)
val appendThese = List(2,4,6,8)
val lst = List[Double]()
val newList = appendThese.foldLeft(myList)((acc, currItem) => AppendOnTopOfList(acc, currItem))



// accessing elements of a list 
println(">>", myList(0))

(>>,1)


defined function AppendOnTopOfList
myList: List[Int] = List(1, 2, 3, 4)
appendThese: List[Int] = List(2, 4, 6, 8)
lst: List[Double] = List()
newList: List[Int] = List(1, 2, 3, 4, 1, 1, 1, 1)

# Compiling Expressions to a List of ByteCode Instructions

We will now describe the compilation of expressions into byteCode Instructions. 

For instance the expression 1.0 + (2.0 - 3.0) * 5.0 is represented as an AST 
    Plus(Const(1.0), Multi(Minus(Const(2.0), Const(3.0)), Const(5.0))
    
The overall goal of this part is to compile this down into a list of bytecode instructions that serves to evaluate this minor emulator oyu have built in part 1 

__The expression above should produce the instructions__ 

    Push(1.0)
    Push(2.0)
    Push(3.0)
    MinusI
    PushI(5.0)
    MultI
    AddI
    
you should check that evaluating this sequence results in -4.0. Please pay particular attention to the order of operands for MinusI according to the specification provided in Part 1. 

The idea is to implement a function __compileExpr(e)__ that given a expression __e__ yields a lsit of instructions according to the following operational semantics definition. 



$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;\; (\textit{#3})} $$
$$\newcommand\comp{\textbf{compileExpr}}$$

### Constant Rule

The rule for constants is simple. An expression `Const(f)` compiles to the instruction `PushI(f)`.

$$\semRule{}{\comp(\texttt{Const(f)}) = [ \text{PushI}(f) ] }{const-rule}$$

Note again that $\comp$ maps expressions to _list_ of instructions.

### Add Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Plus(e1, e2}) = ListContatenation(L_1, L_2 , [ AddI ]) }{add-rule}$$

The instructions concatenate the lists $L_1, L_2$ along with the list consisting of a single instruction `[ AddI ]`. Note that the `++` operator in scala implements the list concatenation.

### Subtract Rule

$$\semRule{\comp(\texttt{e1}) = L_1,\ \comp(\texttt{e2}) = L_2}{\comp(\texttt{Minus(e1, e2}) = ListContatenation(L_1, L_2 , [ SubI ]) }{minus-rule}$$

The instructions concatenate the lists $L_1, L_2$ along with the list consisting of a single instruction `[ SubI ]`. 


### Rules for Other expressions

We hope that you will be able to fill in rules for other cases `Mult`, `Div`, `Exp`, `Log`, `Sine` and `Cosine`.




## Instructions for Part 2

The definition of Expression AST is given in the file Expr.scala

Your goal is to implment the compilation routine __compileToStackMachineCode(e: Expr) List[StackMachineInstructions]__ in the file StackMachineCompilation.scala. The function takes in and expression e and outputs a list of stack machine instructions. 